In [2]:
import sys
sys.path.append('..')

In [3]:
from concurrent.futures import ThreadPoolExecutor
from strategy_center.center.mq import MessageQueueSender
from common.constant import (
    Request,
    Direction,
    Offset
)

In [ ]:
# class MessageQueueSender:

#     def __init__(self, user_name, password, host, vhost, account_id):
#         self.user_name = user_name
#         self.password = password
#         self.host = host
#         self.vhost = vhost
#         self.account_id = account_id
#         self.connection = None
#         self.channel = None
#         self.ensure_connection()

#     def ensure_connection(self):
#         """确保 RabbitMQ 连接是开启的，如果没有则创建它。"""
#         if not self.connection or self.connection.is_closed:
#             self.create_connection()
#         if not self.channel or self.channel.is_closed:
#             self.channel = self.connection.channel()
#             queue_name = f'req{self.account_id}'
#             self.channel.queue_declare(queue=queue_name, durable=True)

#     def create_connection(self):
#         """创建到 RabbitMQ 的连接。"""
#         credentials = pika.PlainCredentials(self.user_name, self.password)
#         parameters = pika.ConnectionParameters(self.host, 5672, self.vhost, credentials)
#         self.connection = pika.BlockingConnection(parameters)

#     def send(self, request):
#         """发送消息，如果连接关闭则尝试重新连接。"""
#         try:
#             self.ensure_connection()
#             queue_name = f'req{self.account_id}'
#             message = json.dumps(dataclass_to_dict(request)).encode('utf-8')

#             self.channel.basic_publish(exchange='',
#                                        routing_key=queue_name,
#                                        body=message)
#             print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f'))
#             print(f" [x] Sent '{message}'")
#         except pika.exceptions.ConnectionClosedByBroker:
#             print("Connection closed by broker, trying to reconnect...")
#             self.connection = None  # 重置连接
#             self.send(request)  # 递归调用自身以重试
#         except pika.exceptions.AMQPChannelError as err:
#             print(f"Channel error: {err}, stopping...")
#         except pika.exceptions.AMQPConnectionError:
#             print("Connection was closed, trying to reopen...")
#             self.create_connection()
#             self.send(request)

#     def close(self):
#         """关闭 RabbitMQ 的连接。"""
#         if self.connection and self.connection.is_open:
#             self.connection.close()

In [4]:
class Trader:
    def __init__(self, user_name, password, host, vhost, account_id):
        self.user_name = user_name
        self.password = password
        self.host = host
        self.vhost = vhost
        self.account_id = account_id
    
    def run(self):
        if not hasattr(self, 'sender'):            
            self.sender = MessageQueueSender(self.user_name, self.password, self.host, self.vhost, self.account_id)
        req = Request(direction=Direction.SHORT, offset=Offset.OPEN, exchange='SHO', code='10006511', amount=30)
        self.sender.send(req)

In [8]:
# traders = [Trader('test', 'test', 'localhost', 'test', '1234') for _ in range(4)]
traders = [Trader('', '', 'localhost', '', '840092285') for _ in range(4)]

# 使用线程池并行运行
with ThreadPoolExecutor(max_workers=8) as executor:
    for trader in traders:
        executor.submit(trader.run)

In [7]:
traders[0].sender.close()
with ThreadPoolExecutor(max_workers=8) as executor:
    for trader in traders:
        executor.submit(trader.run)

2024-02-14 12:08:33.595574
 [x] Sent 'b'{"direction": "\\u5356", "offset": "\\u5f00", "exchange": "SHO", "code": "10006511", "amount": 30, "amount2": 30, "id": "a09980da-996c-4a33-8d65-565a5c0c5bdf"}''
2024-02-14 12:08:33.602651
 [x] Sent 'b'{"direction": "\\u5356", "offset": "\\u5f00", "exchange": "SHO", "code": "10006511", "amount": 30, "amount2": 30, "id": "badb871c-2266-4e61-9d41-6fc6e26a7e65"}''
2024-02-14 12:08:33.602651
 [x] Sent 'b'{"direction": "\\u5356", "offset": "\\u5f00", "exchange": "SHO", "code": "10006511", "amount": 30, "amount2": 30, "id": "11f6fdf5-74df-4905-8147-7a0c035e53eb"}''
2024-02-14 12:08:43.651981
 [x] Sent 'b'{"direction": "\\u5356", "offset": "\\u5f00", "exchange": "SHO", "code": "10006511", "amount": 30, "amount2": 30, "id": "0a3c8c92-4e95-4197-9ccb-8fd957c8ac63"}''
